In [1]:
from ihs_lei.data.load import DatabaseReflector
import pandas as pd

In [2]:
import sqlalchemy
class EconomicDataLoad(object):
    """docstring for EconomicDataLoad"""
    def __init__(self):
        self.db = DatabaseReflector()
        self.economic_table = self.db.reverse_table('t_economics')
        select = sqlalchemy.sql.select([self.economic_table])
        self.economic_data = pd.read_sql(select, self.db.engine)
        

In [3]:
economic_data = EconomicDataLoad()

In [4]:
dataset = economic_data.economic_data
dataset.head()

,quandl_code,date,value,indicator_name,country_name,country_code,indicator_code,start_date,frequency
0,ALBBCONF,2002-06-30,-7.2,Business Confidence,Albania,ALB,BCONF,2002-06-30,Quarterly
1,ALBBCONF,2002-09-30,-2.7,Business Confidence,Albania,ALB,BCONF,2002-06-30,Quarterly
2,ALBBCONF,2002-12-31,3.2,Business Confidence,Albania,ALB,BCONF,2002-06-30,Quarterly
3,ALBBCONF,2003-03-31,1.2,Business Confidence,Albania,ALB,BCONF,2002-06-30,Quarterly
4,ALBBCONF,2003-06-30,-4.1,Business Confidence,Albania,ALB,BCONF,2002-06-30,Quarterly


In [5]:
import ihs_lei
from pathlib import Path
module_init_path = Path(ihs_lei.__file__).parents[1]

In [6]:
countries = dataset[['country_name', 'country_code']]
country_to_country_code = countries.groupby('country_name').first().to_dict()['country_code']

In [20]:
indicator_per_country = dataset[dataset.frequency != 'Yearly']
print(indicator_per_country.head())
indicator_per_country_df = indicator_per_country[['country_name', 'quandl_code']].drop_duplicates().reset_index(drop=True)


  quandl_code        date  value       indicator_name country_name  \
0    ALBBCONF  2002-06-30   -7.2  Business Confidence      Albania   
1    ALBBCONF  2002-09-30   -2.7  Business Confidence      Albania   
2    ALBBCONF  2002-12-31    3.2  Business Confidence      Albania   
3    ALBBCONF  2003-03-31    1.2  Business Confidence      Albania   
4    ALBBCONF  2003-06-30   -4.1  Business Confidence      Albania   

  country_code indicator_code start_date  frequency  
0          ALB          BCONF 2002-06-30  Quarterly  
1          ALB          BCONF 2002-06-30  Quarterly  
2          ALB          BCONF 2002-06-30  Quarterly  
3          ALB          BCONF 2002-06-30  Quarterly  
4          ALB          BCONF 2002-06-30  Quarterly  


In [23]:
indicator_per_country = {}
for country in country_to_country_code.keys():
    indicator_per_country[country] = sorted(set(indicator_per_country_df[indicator_per_country_df['country_name'] == country]['quandl_code']))

In [73]:
freq_dict = {
    'Quarterly': 'Q',
    'Monthly': 'M',
    'Weekly': 'W',
    'Daily': 'B'
}

In [74]:
reference_series_list = ['IP', 'OIL']

In [80]:
from collections import defaultdict
from ihs_lei.analytics.filtering import *

indicator_fit_per_country = defaultdict(lambda: dict())
indicator_model_per_country = defaultdict(lambda: dict())
for country in sorted(country_to_country_code.keys()):
    print('[INFO] Working on {}'.format(country))
    country_exogenous_vars = indicator_per_country[country].copy()
    country_endog_vars = []
    for ref_series in reference_series_list:
        if ref_series in country_exogenous_vars:
            country_endog_vars.append(ref_series)
            country_exogenous_vars.remove(ref_series)
    for indicator_code in country_exogenous_vars:
        try:
            indicator_series = dataset[dataset['quandl_code'] == indicator_code]
            frequency = dataset[dataset['quandl_code'] == indicator_code]['frequency'].values[0]
            indicator_series.index = pd.to_datetime(indicator_series['date'])
            indicator_series = indicator_series['value'].asfreq(freq_dict[frequency], method='ffill').dropna()
            indicator_model_per_country[country][indicator_code] = SeasonalAdjustment(indicator_series)
            indicator_fit_per_country[country][indicator_code] = indicator_model_per_country[country][indicator_code].fit()
        except:
            print('[ERROR] Error in {} in indicator {}'.format(country, indicator_code))

[INFO] Working on Afghanistan
[ERROR] Error in Afghanistan in indicator AFGGOLD
[INFO] Working on Albania
[ERROR] Error in Albania in indicator ALBBCONF
[ERROR] Error in Albania in indicator ALBCNCN
[ERROR] Error in Albania in indicator ALBGAGR
[ERROR] Error in Albania in indicator ALBGCP
[ERROR] Error in Albania in indicator ALBGGR
[ERROR] Error in Albania in indicator ALBGOLD
[ERROR] Error in Albania in indicator ALBIP
[INFO] Working on Algeria
[ERROR] Error in Algeria in indicator DZAFINF
[ERROR] Error in Algeria in indicator DZAGOLD
[ERROR] Error in Algeria in indicator DZAIMPX
[ERROR] Error in Algeria in indicator DZAIP
[ERROR] Error in Algeria in indicator DZAPPI
[INFO] Working on Andorra
[INFO] Working on Angola
[ERROR] Error in Angola in indicator AGOBCONF
[ERROR] Error in Angola in indicator AGOBOT
[ERROR] Error in Angola in indicator AGOCPI
[ERROR] Error in Angola in indicator AGOCPICM
[ERROR] Error in Angola in indicator AGOEXVOL
[ERROR] Error in Angola in indicator AGOIMVOL

In [83]:
print(indicator_series.values.reshape((len(indicator_series.values),)))
print(dataset[dataset['quandl_code'] == 'AGOIP'])

[  297626.   383191.   399843.   495410.   544914.   711303.   784883.
   862033.   969497.   991698.  1210200.  1381250.  1407630.  1545870.
  1689120.  1752270.  1832430.  1850580.  1917670.  2040170.  2289510.
  2151520.  2296300.  2327610.  2361940.  2458120.  2578180.  2601420.
  2733650.  2899720.  2907000.  2952440.  3029290.  3053210.  3088580.
  3100400.  3145020.  3377570.  3438620.  3453760.  3580180.  3590270.
  3697560.  3589300.  3728180.  3813960.  3824660.  3886670.  3808400.
  3813620.  3798520.  3966740.  4018140.  3838210.  3854920.  3796240.
  3910660.  3951680.  3807110.  3932320.  3888640.  4021780.  4093940.
  4230460.  4325730.  4323570.  4224070.  4322080.  4388500.  4457260.
  4415470.  4403120.  4340120.  4337890.  4370320.  4438840.  4487970.
  4544010.  4473960.  4473050.  4586020.  4600830.  4745180.  4765420.
  4754440.]
      quandl_code        date  value         indicator_name country_name  \
14784       AGOIP  2011-03-31   -7.4  Industrial Production 

In [84]:
print(dataset[dataset['quandl_code'] == 'AGOIP'])

      quandl_code        date  value         indicator_name country_name  \
14784       AGOIP  2011-03-31   -7.4  Industrial Production       Angola   
14785       AGOIP  2011-06-30   -9.0  Industrial Production       Angola   
14786       AGOIP  2011-09-30    1.1  Industrial Production       Angola   
14787       AGOIP  2011-12-31    6.3  Industrial Production       Angola   
14788       AGOIP  2012-03-31    6.0  Industrial Production       Angola   
14789       AGOIP  2012-06-30    7.0  Industrial Production       Angola   
14790       AGOIP  2012-09-30    8.8  Industrial Production       Angola   
14791       AGOIP  2012-12-31    1.7  Industrial Production       Angola   
14792       AGOIP  2013-03-31    3.1  Industrial Production       Angola   
14793       AGOIP  2013-06-30    3.4  Industrial Production       Angola   
14794       AGOIP  2013-09-30    2.3  Industrial Production       Angola   
14795       AGOIP  2013-12-31   -2.2  Industrial Production       Angola   
14796       

In [88]:
indicator_fit_to_pickle = {}
for country in indicator_fit_per_country.keys():
    print('[INFO] Country {} has {} indicators fit.'.format(country, len(indicator_fit_per_country[country])))
    indicator_fit_to_pickle[country] = (dict(indicator_fit_per_country[country]), dict(indicator_model_per_country[country]))

[INFO] Country Algeria has 8 indicators fit.
[INFO] Country Lebanon has 14 indicators fit.
[INFO] Country Lithuania has 36 indicators fit.
[INFO] Country Afghanistan has 3 indicators fit.
[INFO] Country Guinea has 6 indicators fit.
[INFO] Country Cuba has 1 indicators fit.
[INFO] Country Saudi Arabia has 16 indicators fit.
[INFO] Country Denmark has 55 indicators fit.
[INFO] Country Switzerland has 47 indicators fit.
[INFO] Country Paraguay has 6 indicators fit.
[INFO] Country Iceland has 29 indicators fit.
[INFO] Country Austria has 47 indicators fit.
[INFO] Country Seychelles has 2 indicators fit.
[INFO] Country Malaysia has 30 indicators fit.
[INFO] Country Argentina has 33 indicators fit.
[INFO] Country Chile has 34 indicators fit.
[INFO] Country Georgia has 7 indicators fit.
[INFO] Country Luxembourg has 40 indicators fit.
[INFO] Country Zambia has 4 indicators fit.
[INFO] Country Serbia has 17 indicators fit.
[INFO] Country Portugal has 46 indicators fit.
[INFO] Country Tajikista

In [ ]:
import pickle
with open('indicator_fit.pkl', 'wb+') as f:
    pickle.dump(indicator_fit_to_pickle, f)